<div style="font-size: 200%; font-weight: bold; color: maroon;">Clustering via k-means - offline</div>
<!-- Paulo Villegas, 2022 - v. 1.5 -->

In [ ]:
%matplotlib inline

In [ ]:
import os.path
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

plt.rcParams['figure.figsize'] = (15, 5)

# 1. Carga de datos

## 1.1 Leer datos

In [ ]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import StringType, FloatType, TimestampType
import pyspark.sql.functions as F

In [ ]:
# Define el esquema
fields = [ StructField('localidad', StringType(), True),
           StructField('hora', StringType(), True),
           StructField('temperatura', FloatType(), True),
           StructField('humedad', FloatType(), True),
           StructField('presion', FloatType(), True),
           StructField('viento', FloatType(), True) ]
          
schema = StructType(fields)

# Carga los datos
meteo = spark.read.format("csv") \
             .option("header","false") \
             .schema(schema) \
             .load("../data/data-meteo.csv.bz2")

In [ ]:
meteo.printSchema()

In [ ]:
meteo = meteo.withColumn('tstamp', F.to_timestamp(F.col('hora'), "dd-MM-yyyy HH:mm z")).drop('hora')

## 1.2 Inspección

In [ ]:
# ¿Cuantos tenemos?
meteo.count()

In [ ]:
meteo.sample(False, 0.0001).toPandas()

In [ ]:
# Rango temporal
meteo.select( F.min('tstamp'), F.max('tstamp') ).show()

In [ ]:
hour = meteo.select(F.hour('tstamp').alias('hora'))

hour.select(F.min('hora'), F.max('hora')).show()

In [ ]:
byhour = meteo.groupBy(F.hour('tstamp').alias('hour')).count()
byhour.orderBy('hour').toPandas().plot(x='hour', kind='bar', legend=None);

## 1.3 Preproceso 1

(eliminación de valores extremos)

In [ ]:
meteo.filter( meteo['presion'] > 1200 ).toPandas()

In [ ]:
meteo = meteo.filter( meteo.presion < 1200)

In [ ]:
meteo.sort('viento', ascending=False).limit(5).toPandas()

In [ ]:
meteo = meteo.filter( meteo.viento < 100)

In [ ]:
meteo.sort('viento', ascending=False).limit(5).toPandas()

# 2. Preprocesado

## 2.1 Creación de la columna de datos de entrada

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["temperatura", "humedad", "presion", "viento"], outputCol="features")

meteo_c = assembler.transform(meteo)

In [ ]:
meteo_c.printSchema()

In [ ]:
meteo_c.limit(5).toPandas()

## 2.2 Estadísticas

In [ ]:
from pyspark.ml.stat import Summarizer

metrics = ("min", "mean", "max", "variance")
summarizer = Summarizer.metrics(*metrics)

In [ ]:
stats = meteo_c.select(summarizer.summary(meteo_c.features).alias('metrics')).toPandas()

In [ ]:
with pd.option_context('display.max_colwidth', 400):
    display(stats)

Como es difícil ver nada con este formato, vamos a recolocar los valores

In [ ]:
import pandas as pd
from typing import List

def reshape_metrics(stats: pd.DataFrame, metric_names: List[str]) -> pd.DataFrame:
    s1 = stats.metrics.apply(pd.Series)
    s1.columns = metric_names
    s2 = s1.applymap(lambda x: x.toArray())
    s3 = s2.melt()
    
    s4 = pd.concat((s3.drop(columns='value'), s3.value.apply(pd.Series)), axis='columns')
    s4.columns = ['stat', 'temperatura', 'humedad', 'presion', 'viento']
    return s4

In [ ]:
m = reshape_metrics(stats, metrics)
m

In [ ]:
import numpy as np

# Desviación estándar
np.sqrt(m.drop(columns='stat').loc[m.stat == 'variance'])

## 2.1 Preproceso 2

(normalización)

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

#meteo_c = meteo_c.drop('scaledFeatures')

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(meteo_c)

# Normalize each feature to have unit standard deviation.
meteo_c = scalerModel.transform(meteo_c)

In [ ]:
summarizer = Summarizer.metrics(*metrics)
stats = meteo_c.select(summarizer.summary(meteo_c.scaledFeatures).alias('metrics')).toPandas()
reshape_metrics(stats, metrics)

# 3. Clustering

In [ ]:
%%time

from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(3).setInitSteps(49)
#kmeans.setFeaturesCol('scaledFeatures')  # "scaledFeatures" para usar las variables normalizadas
model = kmeans.fit(meteo_c)

In [ ]:
# ¿cuántos items han caído en cada cluster?
if model.hasSummary:
    print(model.summary.clusterSizes)

In [ ]:
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
# Escribe los centroides en formato de código
import numpy as np
a = np.array( centers )
repr(a.tolist())

In [ ]:
# Graba el modelo a disco
model.write().overwrite().save("kmeans-model")

In [ ]:
from pyspark.ml.clustering import KMeansModel

# Lee el modelo de disco
m2 = KMeansModel.load("kmeans-model")
m2.clusterCenters()

## 3.1 Inspección de resultados

In [ ]:
# Aplicamos los clusters a todos nuestros datos
meteo_t = model.transform( meteo_c )

In [ ]:
# Tamaño de los clusters
meteo_t.groupBy('prediction').count().show()

In [ ]:
# una muestra
meteo_t.sample(False,0.001).limit(10).toPandas()

In [ ]:
# muestra de datos de un cluster
meteo_t.filter(meteo_t['prediction'] == 2).select('localidad','tstamp').show(10,truncate=False)

In [ ]:
# descargo a local las asignaciones a cluster y las variables de entrada
meteo_data = meteo_t.select( 'temperatura', 'humedad', 'presion', 'viento', 'prediction').toPandas()

meteo_data.dtypes

In [ ]:
%%time

# pintamos su distribución con respecto a las variables de entrada
import seaborn as sns 

sns.pairplot(vars=["temperatura", "humedad", "presion", "viento"], 
             data=meteo_data, hue="prediction", height=5, plot_kws={'alpha': 0.1});